In [1]:
import boto3
import numpy as np
import os
import scipy as sp
from scipy.sparse import csr_matrix
from scipy.sparse import vstack as vstack
from sklearn.preprocessing import MultiLabelBinarizer
import time

In [ ]:
# get files from s3
# s3_client = boto3.client('s3')
# s3_client.download_file('brianlubeck2', 'chess/data/clean/xTrain0.npz', '/home/ec2-user/chess/data/clean/xTrain0.npz')
# s3_client.download_file('brianlubeck2', 'chess/data/clean/xTrain1.npz', '/home/ec2-user/chess/data/clean/xTrain1.npz')
# s3_client.download_file('brianlubeck2', 'chess/data/clean/xTrain2.npz', '/home/ec2-user/chess/data/clean/xTrain2.npz')
# s3_client.download_file('brianlubeck2', 'chess/data/clean/xTrain3.npz', '/home/ec2-user/chess/data/clean/xTrain3.npz')
# s3_client.download_file('brianlubeck2', 'chess/data/clean/yTrain0.npz', '/home/ec2-user/chess/data/clean/yTrain0.npz')
# s3_client.download_file('brianlubeck2', 'chess/data/clean/yTrain1.npz', '/home/ec2-user/chess/data/clean/yTrain1.npz')
# s3_client.download_file('brianlubeck2', 'chess/data/clean/yTrain2.npz', '/home/ec2-user/chess/data/clean/yTrain2.npz')
# s3_client.download_file('brianlubeck2', 'chess/data/clean/yTrain3.npz', '/home/ec2-user/chess/data/clean/yTrain3.npz')

In [2]:
# constants
BOARD_LENGTH = 768 #chess board is 8 x 8 and 12 different pieces

## Vector representation of chess board
# v = 1 x BOARD_LENGTH
#
# White = Upper Case, black = lower case
# Piece order: P, N, B, R, Q, K, p, n, b, r, q, k
# Board order:
#    Start at square a1. Move across the columns to square h1.
#    Then go up a row to square a2. Move across the columns to square h2.
#    Repeat until square h8
#    i.e. 0 - a1, 1 - b1, ..., 7 - h1, 8 - a2, ..., 63 - h8
#
# Board vector indices: 
# v[0,...,63] = P, v[64,...,127] = N, ..., v[704,...,767] = k
# v[0,...,7] = row 1; v[8,...,15] = row 2, ..., v[56,...,63] = row 8
# v[0] = col a, v[1] = col b, ..., v[7] = col h

PIECE_OFFSETS = {'P': 0, 'N': 64, 'B': 128, 'R': 192, 'Q': 256, 'K': 320,
                 'p': 384, 'n': 448, 'b': 512, 'r': 576, 'q': 640, 'k': 704}

RESULTS_DICT = {'1-0': 1,'1/2-1/2': 0,'0-1': -1}
RESULTS_LIST = [1, 0, -1]

# Neural Net

In this section, we build, train and test a feedforward neural network that calculates the probability a board position results in a win for white. We first use a small sample of games to determine a reasonable number of hidden nodes fro the neural network. The program ParseData.py converts the pgn data into binary vectors and saves the output. The program PrepareTrainTest.py takes the binary vectors and creates four training datasets and one testing dataset. We split the data up so that the data could be processed by different cores using mini-batch parallelism. We compare the time to train the neural network of our implementation of mini-batch parallelism versus none.

## Neural Net Class

The code uses sparse matrices to siginificantly decrease the amount of memory and instructions needed on a CPU to train the neural net

In [ ]:
class BoardFunction:
    
    def __init__(self, numInputNodes, numHiddenNodes, numOutputNodes, maxIter, maxEpochs):
        # layers
        self.numInputNodes = numInputNodes
        self.numHiddenNodes = numHiddenNodes
        self.numOutputNodes = numOutputNodes
        
        # weight matrices
        self.hiddenWeights = np.empty((self.numInputNodes, self.numHiddenNodes), dtype = np.float_)
        self.hiddenBiases = np.empty((1, self.numHiddenNodes), dtype = np.float_)
        self.outputWeights = np.empty((self.numHiddenNodes, self.numOutputNodes), dtype = np.float_)
        self.outputBiases = np.empty((1, self.numOutputNodes), dtype = np.float_)
        
        # learning parameters
        self.learningRate = 0.1
        self.minRate = 0.00001
        self.miniBatchSize = 512
        self.maxIter = maxIter
        self.maxEpochs = maxEpochs
        self.minTol = 10**(-7)
        self.decay = True # if true decreses the learning rate if loss plateaus
        self.logPath = '../log'
        
    def initWeights(self, seed = None):
        '''
        Randomly initializes the weight matrices
        '''
        if seed != None:
            np.random.seed(seed)
            
        self.hiddenWeights = np.random.normal(size = self.hiddenWeights.shape)
        self.hiddenBiases = np.random.normal(size = self.hiddenBiases.shape)
        self.outputWeights = np.random.normal(size = self.outputWeights.shape)
        self.outputBiases = np.random.normal(size = self.outputBiases.shape)
        
    def _relu(self, X):
        '''
        X - matrix
        
        returns element wise max of X and zero
        '''
        
        return(np.maximum(X,0))
    
    def _softmax(self, X):
        shiftX = X - np.amax(X, axis = 1, keepdims = True)
        exps = np.exp(shiftX)
        sums = np.sum(exps, axis = 1, keepdims = True)
        
        return(exps / sums)
    
    def predict(self, board):
        '''
        board - csr matrix: sparse row matrix of encoded board positions
    
        returns probs - numpy array: a matrix containing the probability of a win, draw or loss
        '''
        
        numBoards = board.shape[0]
        
        hiddenWeights = board.dot(self.hiddenWeights)
        hiddenBiases = np.outer(np.ones((numBoards, 1), dtype = np.float_), self.hiddenBiases)
        hiddenIn = hiddenWeights + hiddenBiases
        hiddenOut = self._relu(hiddenIn) #rectified linear element-wise max with zero
        
        outputWeights = hiddenOut.dot(self.outputWeights)
        outputBiases = np.outer(np.ones((numBoards, 1), dtype = np.float_), self.outputBiases)
        outputIn = outputWeights + outputBiases
        outputOut = self._softmax(outputIn)
        
        minProb = np.finfo(np.float64).tiny # avoid numerical issues with zero probs
        
        return(np.maximum(outputOut, minProb))
    
    def loss(self, board, result):
        '''
        board - csr matrix: sparse row matrix of encoded board positions
        result - 1d array: one hot enconding of result
        
        returns the cross entropy (multinomial log-likelihood) for the sample
        '''
        
        probs = self.predict(board)
        aveLogLikelihood = -np.sum(result * np.log(probs)) / board.shape[0]
        
        return(aveLogLikelihood)
    
    def calcGradients(self, board, result):
        '''
        board - csr matrix: sparse row matrix of encoded board positions
        result - 1d array: one hot enconding of result
        
        J = cross entropy loss function
        '''
        
        numBoards = board.shape[0]
        
        # feed forward
        hiddenWeights = board.dot(self.hiddenWeights)
        hiddenBiases = np.outer(np.ones((numBoards, 1), dtype = np.float_), self.hiddenBiases)
        hiddenIn = hiddenWeights + hiddenBiases
        hiddenOut = self._relu(hiddenIn) #rectified linear element-wise max with zero
        
        outputWeights = hiddenOut.dot(self.outputWeights)
        outputBiases = np.outer(np.ones((numBoards, 1), dtype = np.float_), self.outputBiases)
        outputIn = outputWeights + outputBiases
        outputOut = self._softmax(outputIn)
        
        # compute gradients
        d1 = outputOut - result
        d2 = d1.dot(self.outputWeights.transpose()) * np.sign(hiddenOut)
        
        # D J(outputWeights)
        DJoutW = hiddenOut.transpose().dot(d1) / numBoards
        
        # D J(outputBiases)
        DJoutB = np.sum(d1.dot(np.eye(result.shape[1])), axis = 0) / numBoards
        
        # D J(hiddenWeights)
        DJhidW = board.transpose().dot(d2) / numBoards
        
        # D J(hiddenBiases)
        DJhidB = np.sum(d2, axis = 0) / numBoards
        
        return(DJoutW, DJoutB, DJhidW, DJhidB)
    
    def saveIter(self, fileName, fileNum, numEpoch, numIter, loss):
        line = str(fileNum) + ',' + str(numEpoch) + ',' + str(numIter) + ',' + str(loss) + '\n'
        with open(fileName, mode ='a') as f:
            f.write(line)
    
    def saveWeights(self, fileName):
        print('Saving weights to ' + fileName)
        np.savez_compressed(fileName,
                            hiddenWeights = self.hiddenWeights,
                            hiddenBiases = self.hiddenBiases,
                            outputWeights = self.outputWeights,
                            outputBiases = self.outputBiases)
        print('Done saving weights')
        
    def applyGradients(self, DJoutW, DJoutB, DJhidW, DJhidB):
        '''
        
        updates each weight matrix by subtracting off the learning rate times the gradient
        '''
        
        self.hiddenWeights = self.hiddenWeights - self.learningRate * DJhidW
        self.hiddenBiases = self.hiddenBiases - self.learningRate * DJhidB
        self.outputWeights = self.outputWeights - self.learningRate * DJoutW
        self.outputBiases = self.outputBiases - self.learningRate * DJoutB
    
    def train(self, xTrain, yTrain, logFile, fileNum):
        
        # setup training
        resultOneHot = MultiLabelBinarizer(classes = RESULTS_LIST)
        batchFeeder = self.nextBatch(xTrain.shape[0], self.miniBatchSize)
    
        numIter = 0
        changedEpoch = 1
        loss = np.zeros(10, dtype=np.float_) # saves the 10 previous checkpoint losses
        stop = False
        
        print('Start training hidden nodes ' + str(self.numHiddenNodes))
        print('Learning rate is {}'.format(self.learningRate))
    
        while stop == False:
            # get batch
            firstInd, lastInd, numEpoch = next(batchFeeder)
            board = xTrain[firstInd:lastInd]
            result = resultOneHot.fit_transform(yTrain[firstInd:lastInd])
            
            # calc and save loss every 50 iterations
            if numIter % 50 == 0:
                i = (numIter // 50) % 10
                loss[i] = self.loss(board, result)
                #print('Iteration: {0} Loss: {1}'.format(numIter, loss[i]))
                self.saveIter(logFile, fileNum, numEpoch, numIter, loss[i])
        
            # calc and apply gradients
            DJoutW, DJoutB, DJhidW, DJhidB = self.calcGradients(board, result)
            self.applyGradients(DJoutW, DJoutB, DJhidW, DJhidB)
        
            # update number of iterations
            numIter = numIter + 1
        
            # check if max number of iterations or epochs
            if numIter > self.maxIter or numEpoch > self.maxEpochs:
                stop = True
                self.saveIter(logFile, fileNum, numEpoch - 1, numIter - 1, loss[i])
                print('Stopped training at {} iterations and {} epochs for hidden nodes {}'.format(
                    numIter, numEpoch, self.numHiddenNodes))
                
            # if decay is true change learning rate at epoch
            if self.decay == True and numEpoch - changedEpoch > 10:
                changedEpoch = numEpoch
                self.learningRate = self.learningRate / 10
                print('Epoch {}. Changed learning rate to {}'.format(numEpoch, self.learningRate))

    
    def nextBatch(self, totObs, batchSize):
        # initialize
        numBatches = totObs // batchSize
        tail = totObs % batchSize
        batch = 0
        epoch = 0
    
        # generator
        while True:
            batch = (batch + 1) % numBatches
            if batch == 1:
                firstInd = 0
                lastInd = batchSize
                epoch = epoch + 1
            elif batch == 0:
                firstInd = lastInd
                lastInd = lastInd + batchSize + tail
            else:
                firstInd = lastInd
                lastInd = lastInd + batchSize
        
            yield firstInd, lastInd, epoch

## Neural Net of Full Data

Based on sample data results, it appears that 100 is a reasonable number of hidden nodes for 294,428 total board positions in the data (of which 261,013 are unique)

In [3]:
dataDir = '../data/clean'
xTrainFiles = [os.path.join(dataDir, 'xTrain' + str(i) + '.npz') for i in range(4)]
yTrainFiles = [os.path.join(dataDir, 'yTrain' + str(i) + '.npz') for i in range(4)]

# xTrain = sp.sparse.load_npz(xTrainFiles[0])
# yTrain = np.load(yTrainFiles[0])['arr_0']

In [ ]:
# test run
logFile = '../log/model_full_log.txt'
modelFile = '../model/model_full_weights.npz'

# def __init__(self, numInputNodes, numHiddenNodes, numOutputNodes, maxIter, maxEpochs):
boardFunc = BoardFunction(BOARD_LENGTH, 32000, 3, 200, 1)
boardFunc.initWeights()

# train on data chunk
startTime = time.time()
boardFunc.train(xTrain, yTrain, logFile, 0)
trainTime = time.time() - startTime
print(trainTime)

In [ ]:
# 32000 hidden nodes, 100 iterations through each chunk
# baseline training time
# load through data files and train
trainTime = 0
for fx, fy, fileNum in zip(xTrainFiles, yTrainFiles, range(4)):
    # load data file
    xTrain = sp.sparse.load_npz(fx)
    yTrain = np.load(fy)['arr_0']
    
    # train on data chunk
    print('File ' + str(fileNum))
    startTime = time.time()
    boardFunc.train(xTrain, yTrain, logFile, fileNum)
    trainTime = trainTime + time.time() - startTime
    # save parameters
    # boardFunc.saveWeights(modelFile)
    # change learning rate
    # boardFunc.learningRate = boardFunc.learningRate / 10
    print()
print(trainTime)

## MXNET Implementation

Gloun tutorial at http://gluon.mxnet.io/chapter03_deep-neural-networks/mlp-gluon.html

In [4]:
import mxnet as mx
from mxnet import nd, autograd, gluon, io

In [5]:
# model context
dataCtx = mx.cpu()
modelCtx = mx.cpu()

# architecture
numInputs = BOARD_LENGTH
numHidden = 32000
numOutputs = 3

net = gluon.nn.Sequential()
with net.name_scope():
    dense0 = net.add(gluon.nn.Dense(numHidden, activation = 'relu'))
    dense1 = net.add(gluon.nn.Dense(numOutputs))

# parameter initialization
net.collect_params().initialize(mx.init.Normal(sigma=0.1), ctx=modelCtx)

# softmax cross-entropy loss
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss(sparse_label = False)

# optimizer
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.01})        

In [ ]:
dataDir = '../data/clean'
xTrainFiles = [os.path.join(dataDir, 'xTrain' + str(i) + '.npz') for i in range(4)]
yTrainFiles = [os.path.join(dataDir, 'yTrain' + str(i) + '.npz') for i in range(4)]

# training loop

logFile = '../log/model_full_log.txt'
modelFile = '../model/model_full_weights'

epochs = 4
iterNum = 0
for epoch in range(epochs):
    print('Epoch ' + str(epoch))
    for fx, fy, fileNum in zip(xTrainFiles, yTrainFiles, range(4)):
        # load data file
        print('File ' + str(fileNum))
        print('Loading file ' + str(fileNum))
        xTrain = sp.sparse.load_npz(fx)
        yTrain = np.load(fy)['arr_0']
        print('Done loading file')

        # Create batch iterator
        # convert xTrain and yTrain to mxnet formats
        # scipy csr format to mxnet csr format
        print('Formating data for MXnet')
        xTrainMX = nd.sparse.csr_matrix((xTrain.data, xTrain.indices, xTrain.indptr), dtype = 'float32')
        yTrainMX = 1 - nd.array(np.squeeze(yTrain), dtype = 'float32') # white win = 0, draw = 1, black win = 2

        trainData = io.NDArrayIter(data = xTrainMX, label = nd.one_hot(yTrainMX, depth = 3),
                                    batch_size = 512, last_batch_handle = 'discard')
        # trainData = io.NDArrayIter(data = xTrainMX, label = np.squeeze(yTrain).astype(np.float32),
        #                            batch_size = 512, last_batch_handle = 'discard')
        print('Done formating data')

        print('Start training')
        startTime = time.time()
        for batch in trainData:
            X = batch.data[0].as_in_context(modelCtx)
            y = batch.label[0].as_in_context(modelCtx)
            with autograd.record():
                output = net(X)
                loss = softmax_cross_entropy(output, y)
            loss.backward()
            trainer.step(512)
            l = nd.sum(loss).asscalar()
            if iterNum % 100 == 0:
                line = str(fileNum) + ',' + str(epoch) + ',' + str(iterNum) + ',' + str(l) + '\n'
                with open(logFile, mode = 'a') as f:
                    f.write(line)
            if iterNum % 20000 == 0:
                'checkpoint parameters'
                net.save_params(modelFile)
            iterNum = iterNum + 1
        
        endTime = time.time()
        print('Done training file in ' + str(endTime - startTime))

In [7]:
# load model params
modelFile = '../model/model_full_weights'
net.load_params('../model/model_full_weights', modelCtx)

In [8]:
params = net.collect_params()

In [13]:
weights = []
for param in params.values():
    #print(param.name, param.data())
    weights.append(param.data().asnumpy())

In [20]:
np.savez_compressed('../model/model_full_weights.npz',
                            hiddenWeights = weights[0],
                            hiddenBiases = weights[1],
                            outputWeights = weights[2],
                            outputBiases = weights[3])

In [19]:
weights[3].shape

(3,)

In [ ]:

batch_size = 64
num_inputs = 784
num_outputs = 10
num_examples = 60000
def transform(data, label):
    return data.astype(np.float32)/255, label.astype(np.float32)
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),
                                     batch_size, shuffle=False)

In [ ]:
data_ctx = mx.cpu()
model_ctx = mx.cpu()

batch_size = 64
num_inputs = 784
num_outputs = 10
num_examples = 60000
def transform(data, label):
    return data.astype(np.float32)/255, label.astype(np.float32)
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),
                              batch_size, shuffle=False)

In [ ]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(model_ctx).reshape((-1,784))
        label = label.as_in_context(model_ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

net = gluon.nn.Dense(num_outputs)
net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=model_ctx)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

evaluate_accuracy(test_data, net)

In [ ]:
epochs = 10
moving_loss = 0.

for e in range(epochs):
    cumulative_loss = 0
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(model_ctx).reshape((-1,784))
        label = label.as_in_context(model_ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        cumulative_loss += nd.sum(loss).asscalar()

    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, cumulative_loss/num_examples, train_accuracy, test_accuracy))

In [ ]:
label